In [2]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.5 MB/s eta 0:00:00a 0:00:01


In [13]:
import os
import time
from datetime import datetime
from groq import Groq
import math

In [6]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [16]:
def measure_feedback_latency(questions, score, time_elapsed):
    try:
        # Start timing
        start_time = time.time()
        
        # Round up the time elapsed
        rounded_time = math.ceil(time_elapsed)
        from groq import Groq
        # Generate the feedback
        client = Groq(api_key="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")
        completion = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[
                {
                    "role": "system", 
                    "content":f"""
                         A user has completed a quiz with the following statistics:
        
                        - Total Questions: {questions}
                        - Correct Answers: {score}
                        - Time Taken: {rounded_time} seconds
        
                        Based on the user's performance, provide constructive feedback on their quiz attempt. 
                        Consider the user's score and the time taken. Offer encouragement and suggest areas for improvement.
                        Please keep feedback short and sweet.
                        """
                },
                {"role": "user", "content": f"#Questions:{questions},#Score:{score},#time taken:{rounded_time},"},
            ],
            max_tokens=150,
            temperature=0.5,
            top_p=0.9
        )
        
        # Calculate latency
        end_time = time.time()
        latency = (end_time - start_time) * 1000  # Convert to milliseconds
        
        # Get the feedback content
        feedback = completion.choices[0].message.content
        
        return {
            'feedback': feedback,
            'latency': latency,
            'timestamp': datetime.now(),
            'success': True
        }
        
    except Exception as e:
        return {
            'feedback': None,
            'latency': None,
            'timestamp': datetime.now(),
            'success': False,
            'error': str(e)
        }


In [17]:
def main():
    # Example values
    questions = 10
    score = 8
    time_elapsed = 300.75  # example time in seconds
    
    result = measure_feedback_latency(questions, score, time_elapsed)
    
    if result['success']:
        print(f"\nFeedback Generated Successfully:")
        print(f"Latency: {result['latency']:.2f} ms")
        print(f"Timestamp: {result['timestamp']}")
        print(f"\nFeedback:\n{result['feedback']}")
    else:
        print(f"Error generating feedback: {result['error']}")

if __name__ == "__main__":
    main()


Feedback Generated Successfully:
Latency: 374.24 ms
Timestamp: 2024-10-02 14:45:05.744795

Feedback:
Great effort on your quiz attempt! You scored 8 out of 10, which is an impressive 80%! You've demonstrated a strong understanding of the material and should be proud of your achievement.

However, there's always room for improvement. Take a closer look at the two questions you got wrong and see if you can identify any common mistakes or areas where you can brush up.

Additionally, consider how you can manage your time more effectively. You took 301 seconds to complete the quiz, which is a bit longer than ideal. Practice pacing yourself and making the most of your time to achieve an even higher score next time.

Keep up the good work and keep pushing yourself to improve!


In [20]:
import statistics

In [21]:
def measure_feedback_latency(questions, score, time_elapsed, num_samples=5):
    latencies = []
    feedbacks = []
    
    print(f"Making {num_samples} API calls to calculate average latency...")
    
    for i in range(num_samples):
        try:
            # Start timing
            start_time = time.time()
            
            # Round up the time elapsed
            rounded_time = math.ceil(time_elapsed)
            
            from groq import Groq
        # Generate the feedback
            client = Groq(api_key=os.getenv("GROQ_API_KEY"))
            
            # Generate the feedback
            completion = client.chat.completions.create(
                model="llama3-8b-8192",
                messages=[
                    {
                        "role": "system", 
                        "content": f"""
                            A user has completed a quiz with the following statistics:
                            - Total Questions: {questions}
                            - Correct Answers: {score}
                            - Time Taken: {rounded_time} seconds
                            Based on the user's performance, provide constructive feedback on their quiz attempt. 
                            Consider the user's score and the time taken. Offer encouragement and suggest areas for improvement.
                            Please keep feedback short and sweet.
                        """
                    },
                    {"role": "user", "content": f"#Questions:{questions},#Score:{score},#time taken:{rounded_time},"},
                ],
                max_tokens=150,
                temperature=0.5,
                top_p=0.9
            )
            
            # Calculate latency
            end_time = time.time()
            latency = (end_time - start_time) * 1000  # Convert to milliseconds
            
            latencies.append(latency)
            feedbacks.append(completion.choices[0].message.content)
            
            print(f"Sample {i+1}: {latency:.2f} ms")
            
            # Add a small delay between requests to avoid rate limiting
            time.sleep(1)
            
        except Exception as e:
            print(f"Error in sample {i+1}: {str(e)}")
            continue
    
    if latencies:
        stats = {
            'average_latency': statistics.mean(latencies),
            'median_latency': statistics.median(latencies),
            'min_latency': min(latencies),
            'max_latency': max(latencies),
            'std_dev': statistics.stdev(latencies) if len(latencies) > 1 else 0,
            'samples_collected': len(latencies),
            'timestamp': datetime.now(),
            'feedback': feedbacks[-1]  # Return the last generated feedback
        }
        return stats
    else:
        return None

In [22]:
def main():
    # Example values
    questions = 10
    score = 8
    time_elapsed = 300.75
    
    # Calculate average latency with multiple samples
    results = measure_feedback_latency(questions, score, time_elapsed, num_samples=5)
    
    if results:
        print("\nLatency Statistics:")
        print(f"Average Latency: {results['average_latency']:.2f} ms")
        print(f"Median Latency: {results['median_latency']:.2f} ms")
        print(f"Minimum Latency: {results['min_latency']:.2f} ms")
        print(f"Maximum Latency: {results['max_latency']:.2f} ms")
        print(f"Standard Deviation: {results['std_dev']:.2f} ms")
        print(f"Samples Collected: {results['samples_collected']}")
        print(f"Timestamp: {results['timestamp']}")
        print(f"\nLast Generated Feedback:\n{results['feedback']}")

if __name__ == "__main__":
    main()

Making 5 API calls to calculate average latency...
Sample 1: 319.45 ms
Sample 2: 279.57 ms
Sample 3: 276.38 ms
Sample 4: 291.65 ms
Sample 5: 306.57 ms

Latency Statistics:
Average Latency: 294.72 ms
Median Latency: 291.65 ms
Minimum Latency: 276.38 ms
Maximum Latency: 319.45 ms
Standard Deviation: 18.22 ms
Samples Collected: 5
Timestamp: 2024-10-02 14:50:27.487183

Last Generated Feedback:
Congratulations on completing the quiz!

You scored an impressive 8/10, which shows you have a strong understanding of the material. Your accuracy is commendable, and you should be proud of your efforts!

However, you might want to focus on improving your time management skills. You took 301 seconds to complete the quiz, which is a bit longer than expected. Practice makes perfect, so try to allocate your time more efficiently to maximize your score.

Keep up the good work, and don't be afraid to challenge yourself with more quizzes!


In [ ]:
def generate_explanation(context ,quiz_data_with_selected_options):
    from groq import Groq
    client = Groq(api_key=os.getenv("GROQ_API_KEY"))
    
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {
                "role": "system", 
                "content": (
                    "Please generate explanation why the selected option is wrong or right based on the question and right option for the given context.\nThe explanation should include complete explanation and avoid cutoff mid-sentence.\nPlease rewrite the following explanation to be clear, concise, and no more than 4-5 sentences.\nUser will provide context, question and right option and selected option in the format as \"#context: particular context, #Question: particular question, #Right option: right option, #Selected option: selected option\". \n"
                )
            },
            {"role": "user", "content": f"#Question: {quiz_data_with_selected_options['question']}, #Right option: {quiz_data_with_selected_options['correct_option']}, #Selected option: {quiz_data_with_selected_options['selected_option']}"},
        ],
        max_tokens=150,  # Lowering the token limit to control length
        temperature=0.5,  # Lowering temperature for less randomness and more focus
        top_p=0.9  # Optional: slightly lower top_p for more coherent completions
    )

    return completion.choices[0].message.content

In [25]:
def measure_feedback_latency(quiz_data_with_selected_options, num_samples=5):
    latencies = []
    feedbacks = []

    print(f"Making {num_samples} API calls to calculate latency...")

    for i in range(num_samples):
        try:
            # Get the question data for the current sample
            question_data = quiz_data_with_selected_options[i % len(quiz_data_with_selected_options)]
            
            # Start timing
            start_time = time.time()

            # Generate the feedback (assuming the API works with the given model)
            client = Groq(api_key=os.getenv("GROQ_API_KEY"))
            completion = client.chat.completions.create(
                model="llama3-8b-8192",
                messages=[
                    {
                        "role": "system",
                        "content": (
                            "Please generate explanation why the selected option is wrong or right based on the question and right option for the given context.\n"
                            "The explanation should include complete explanation and avoid cutoff mid-sentence.\n"
                            "Please rewrite the following explanation to be clear, concise, and no more than 4-5 sentences.\n"
                            "User will provide context, question and right option and selected option in the format as "
                            "\"#context: particular context, #Question: particular question, #Right option: right option, #Selected option: selected option\". \n"
                        )
                    },
                    {
                        "role": "user",
                        "content": f"#Question: {question_data['question']}, #Right option: {question_data['correct_option']}, #Selected option: {question_data['selected_option']}"
                    }
                ],
                max_tokens=150,
                temperature=0.5,
                top_p=0.9
            )

            # Calculate latency
            end_time = time.time()
            latency = (end_time - start_time) * 1000  # Convert to milliseconds
            latencies.append(latency)
            feedbacks.append(completion.choices[0].message.content)

            print(f"Sample {i+1}: {latency:.2f} ms")

            # Add a small delay between requests to avoid rate limiting
            time.sleep(1)

        except Exception as e:
            print(f"Error in sample {i+1}: {str(e)}")
            continue

    if latencies:
        stats = {
            'average_latency': statistics.mean(latencies),
            'median_latency': statistics.median(latencies),
            'min_latency': min(latencies),
            'max_latency': max(latencies),
            'std_dev': statistics.stdev(latencies) if len(latencies) > 1 else 0,
            'samples_collected': len(latencies),
            'timestamp': datetime.now(),
            'feedback': feedbacks[-1]  # Return the last generated feedback
        }
        return stats
    else:
        return None


In [27]:
def main():
    # Example quiz data
    quiz_data_with_selected_options = [
    {
        'question': 'What was the first form of life on Earth?',
        'correct_option': 'Microorganisms',
        'selected_option': 'Plants'
    },
    {
        'question': 'When did humans in their current appearance emerge on Earth?',
        'correct_option': '300,000 years ago',
        'selected_option': '2.5 million years ago'
    },
    {
        'question': 'What is a key aspect of the scientific method?',
        'correct_option': 'Making careful observations and testing hypotheses through experiments',
        'selected_option': 'Drawing conclusions without experimentation'
    },
    {
        'question': 'Which of the following is not considered a recent development in the biological timeline?',
        'correct_option': 'Microorganisms',
        'selected_option': 'Mammals'
    },
    {
        'question': 'What global effort is biology related to in recent news?',
        'correct_option': 'Solving environmental issues and addressing climate change',
        'selected_option': 'Increasing the number of human-made structures'
    },
    {
        'question': 'In which field is the scientific method harder to apply?',
        'correct_option': 'Archaeology and psychology',
        'selected_option': 'Physics and chemistry'
    },
    {
        'question': 'How long have mammals existed on Earth?',
        'correct_option': '130 to 250 million years',
        'selected_option': '300,000 years'
    },
    {
        'question': 'What is a global concern related to biology?',
        'correct_option': 'Outbreaks of Escherichia coli and Salmonella',
        'selected_option': 'Discovery of new chemical elements'
    },
    {
        'question': 'Which of the following diseases is biology actively researching for a cure?',
        'correct_option': 'AIDS, Alzheimer\'s, and cancer',
        'selected_option': 'Common cold'
    },
    {
        'question': 'Why is the scientific method important in biology?',
        'correct_option': 'It helps scientists acquire knowledge through repeatable experiments.',
        'selected_option': 'It allows scientists to make assumptions without experiments.'
    }
]

    
    # Calculate latency for feedback
    results = measure_feedback_latency(quiz_data_with_selected_options, num_samples=5)
    
    if results:
        print("\nLatency Statistics:")
        print(f"Average Latency: {results['average_latency']:.2f} ms")
        print(f"Median Latency: {results['median_latency']:.2f} ms")
        print(f"Minimum Latency: {results['min_latency']:.2f} ms")
        print(f"Maximum (Worst-case) Latency: {results['max_latency']:.2f} ms")
        print(f"Standard Deviation: {results['std_dev']:.2f} ms")
        print(f"Samples Collected: {results['samples_collected']}")
        print(f"Timestamp: {results['timestamp']}")
        print(f"\nLast Generated Feedback:\n{results['feedback']}")
        
if __name__ == "__main__":
    main()

Making 5 API calls to calculate latency...
Sample 1: 327.43 ms
Sample 2: 272.90 ms
Sample 3: 249.91 ms
Sample 4: 300.36 ms
Sample 5: 282.32 ms

Latency Statistics:
Average Latency: 286.58 ms
Median Latency: 282.32 ms
Minimum Latency: 249.91 ms
Maximum (Worst-case) Latency: 327.43 ms
Standard Deviation: 29.19 ms
Samples Collected: 5
Timestamp: 2024-10-02 15:24:28.299590

Last Generated Feedback:
The selected option, "Increasing the number of human-made structures", is incorrect. The right option, "Solving environmental issues and addressing climate change", is correct because recent news has highlighted the importance of biology in addressing global environmental issues and climate change. Biology plays a crucial role in understanding the impact of human activities on the environment and developing sustainable solutions to mitigate climate change.
